# Report Generation Tutorial

PyGSTi is able to construct polished report documents, which provide high-level summaries as well as detailed analyses  of GST results.  Reports are intended to be quick and easy way of analyzing a GST estimate, and pyGSTi's report generation functions are specifically designed to interact with its high-level driver functions (see the high-level algorithms tutorial).  Currently there is only a single report generation function, `pygsti.report.create_general_report`, which takes one or more `Results` objects as input and produces an HTML file as output.  The HTML format allows the reports to include **interactive plots** and **switches**, making it easy to compare different types of analysis or data sets.  

PyGSTi's "general" report creates a stand-alone HTML document which cannot run Python.  Thus, all the results displayed in the report must be pre-computed (in Python).  If you find yourself wanting to fiddle with things and feel that the general report is too static, please consider using a `Workspace` object (see following tutorials) within a Jupyter notebook, where you can intermix report tables/plots and Python.  Internally, `create_general_report` is just a canned routine which uses a `WorkSpace` object to generate various tables and plots and then inserts them into a HTML template.  

**Note to veteran users:** PyGSTi has recently transitioned to producing HTML (rather than LaTeX/PDF) reports.  The way to generate such report is largely unchanged, with one important exception.  Previously, the `Results` object had various report-generation methods included within it.  We've found this is too restrictive, as we'd sometimes like to generate a report which utilizes the results from multiple runs of GST (to compare the, for instance).  Thus, the `Results` class is now just a container for a `DataSet` and its related `GateSet`s, `GatestringStructure`s, etc.  All of the report-generation capability is now housed in within separate report functions, which we now demonstrate.


### Get some `Results`
We start by performing GST using `do_long_sequence_gst`, as usual, to create a `Results` object (we could also have just loaded one from file).

In [1]:
import pygsti
from pygsti.construction import std1Q_XYI

gs_target = std1Q_XYI.gs_target
fiducials = std1Q_XYI.fiducials
germs = std1Q_XYI.germs
maxLengths = [1,2,4,8,16]
ds = pygsti.io.load_dataset("tutorial_files/Example_Dataset.txt")

#Run GST
gs_target.set_all_parameterizations("TP") #TP-constrained
results = pygsti.do_long_sequence_gst(ds, gs_target, fiducials, fiducials, germs,
                                      maxLengths, verbosity=1)

Loading tutorial_files/Example_Dataset.txt: 100%
--- Gate Sequence Creation ---
 1201 sequences created
 Dataset has 3121 entries: 1201 utilized, 0 requested sequences were missing
--- LGST ---
--- Iterative MLGST: [##################################################] 100.0%  1201 gate strings ---
Iterative MLGST Total Time: 1.1s


### Make the reports
Now that we have `results`, we use the methods within `pygsti.report.factory` to generate reports.  These reports are HTML pages that you can open in your browser or email to your friends.  Setting `auto_open=True` makes the finished report open in your web browser automatically.  

In [2]:
pygsti.report.create_general_report(results, "tutorial_files/exampleGenReport.html", 
                                    title="GST Example Report", verbosity=3, auto_open=True)

*** Creating workspace ***
*** Generating switchboard tables ***
*** Generating plots ***
*** Merging into template file ***
Output written to tutorial_files/exampleGenReport.html
Opening tutorial_files/exampleGenReport.html...


Next, let's analyze the same data two different ways: with and without the TP-constraint (i.e. whether the gates *must* be trace-preserving) and furthermore gauge optmimize each case using several different SPAM-weights.  In each case we'll call `do_long_sequence_gst` with `gaugeOptParams=False`, so that no gauge optimization is done, and then perform several gauge optimizations separately and add these to the `Results` object via its `add_gaugeoptimized` function.

In [3]:
#Case1: TP-constrained GST
tpTarget = gs_target.copy()
tpTarget.set_all_parameterizations("TP")
results_tp = pygsti.do_long_sequence_gst(ds, tpTarget, fiducials, fiducials, germs,
                                      maxLengths, gaugeOptParams=False, verbosity=1)

#Gauge optimize
est = results_tp.estimates['default']
gsFinal = est.gatesets['final iteration estimate']
gsTarget = est.gatesets['target']
for spamWt in [1e-4,1e-2,1.0]:
    gs = pygsti.gaugeopt_to_target(gsFinal,gsTarget,{'gates':1, 'spam':spamWt})
    est.add_gaugeoptimized({'itemWeights': {'gates':1, 'spam':spamWt}}, gs, "Spam %g" % spamWt)

--- Gate Sequence Creation ---
 1201 sequences created
 Dataset has 3121 entries: 1201 utilized, 0 requested sequences were missing
--- LGST ---
--- Iterative MLGST: [##################################################] 100.0%  1201 gate strings ---
Iterative MLGST Total Time: 1.1s


In [4]:
#Case2: "Full" GST
fullTarget = gs_target.copy()
fullTarget.set_all_parameterizations("full")
results_full = pygsti.do_long_sequence_gst(ds, fullTarget, fiducials, fiducials, germs,
                                      maxLengths, gaugeOptParams=False, verbosity=1)

#Gauge optimize
est = results_full.estimates['default']
gsFinal = est.gatesets['final iteration estimate']
gsTarget = est.gatesets['target']
for spamWt in [1e-4,1e-2,1.0]:
    gs = pygsti.gaugeopt_to_target(gsFinal,gsTarget,{'gates':1, 'spam':spamWt})
    est.add_gaugeoptimized({'itemWeights': {'gates':1, 'spam':spamWt}}, gs, "Spam %g" % spamWt)

--- Gate Sequence Creation ---
 1201 sequences created
 Dataset has 3121 entries: 1201 utilized, 0 requested sequences were missing
--- LGST ---
--- Iterative MLGST: [##################################################] 100.0%  1201 gate strings ---
Iterative MLGST Total Time: 1.6s


We'll now call the *same* `create_general_report` function but this time instead of passing a single `Results` object as the first argument we'll pass a *dictionary* of them.  This will result in a report that includes switches to select which case ("TP" or "Full") as well as which gauge optimization to display output quantities for.

In [5]:
pygsti.report.create_general_report({'TP': results_tp, "Full": results_full},
                                    "tutorial_files/exampleMultiGenReport.html",
                                    title="Example Multi-Dataset Report", 
                                    verbosity=3, auto_open=True)

*** Creating workspace ***
*** Generating switchboard tables ***
*** Generating plots ***
*** Merging into template file ***
Output written to tutorial_files/exampleMultiGenReport.html
Opening tutorial_files/exampleMultiGenReport.html...


Note that these HTML reports are created quickly but require some time to load within a browser.  This is because the typsetting and figure rendering is done within your browser, rather than up front by `pdflatex`.  Also note that the plots in the new HTML reports are  interactive - a primary advantage of the HTML format.

Other report formats such as **`beamer`-class PDF presentation and Powerpoint presentation have been dropped from pyGSTi**.  These presentation formats were rarely used and moreover we feel that the HTML format is able to provide all of the functionality that was present in these discontinued formats.

### Sharing with others
The report factory functions may be passed a boolean `connected` argument to specify whether online (`True`) or local (`False`) resources should be used by the generated report HTML files.  If `connected=False` (the default, and so what we used in this notebook), then the HTML files rely on the presence of an `offline` folder being in their directory to operate correctly.  The offline folder is automatically generated for you when you call any of the report factory functions, so when viewing reports on the same machine that generates them you typically don't need to worry about it.  

However, *sharing offline reports* (created with `connected=False`) requires that the recipient has pyGSTi's `offline` folder in the same diretory as any report HTML files.  To facilitate sharing, you can get a zipped copy of the offline folder by calling `create_offline_zip` like this:

In [6]:
pygsti.report.create_offline_zip("tutorial_files/")
  #creates .../tutorial_files/offline.zip

If you use `connected=True`, then there's no need for the offline folder -- but you'll need to have an internet connection to load the HTML file.